# GAN

## Import Packages

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras.layers as tfl
import tensorflow.keras.models as tfm
from tensorflow.keras.optimizers.legacy import Adam 
from tensorflow.keras.losses import BinaryCrossentropy
from progressbar import progressbar

## Define Hyperparameters

In [ ]:
LABEL2CLASS = {0: 'T-shirt/top', 
               1: 'Trouser',
               2: 'Pullover',
               3: 'Dress',
               4: 'Coat',
               5: 'Sandal',
               6: 'Shirt',
               7: 'Sneaker',
               8: 'Bag',
               9: 'Ankle Boot'}
HEIGHT = 28
WIDTH = 28
CHANNEL = 1
BATCH_SIZE = 128
LATENT_DIM = 128
EPOCHS = 10

## Load Data

In [ ]:
# load data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train
# gather together all the images
x = np.concatenate([x_train, x_test], 0) ; y = np.concatenate([y_train, y_test], 0)

# add channel dimension
x = np.expand_dims(x, axis = -1)

# scale data
x = x.astype('float64')
y = y.astype('uint8')
x /= 255.0

# make x and y to batches
# x: (number of batches, batch size, height, width, channel)
# y: (number of batches, batch size, 1)
num_of_pics = x.shape[0] ; num_of_batches = int(num_of_pics / BATCH_SIZE)
x_batch = np.zeros((num_of_batches, BATCH_SIZE, HEIGHT, WIDTH, CHANNEL), dtype = np.float64)
y_batch = np.zeros((num_of_batches, BATCH_SIZE), dtype = np.uint8)
for i in range(num_of_batches):
    x_batch[i] = x[i * BATCH_SIZE : (i + 1) * BATCH_SIZE]
    y_batch[i] = y[i * BATCH_SIZE : (i + 1) * BATCH_SIZE]


# convert to tensor
x = tf.convert_to_tensor(x)
x_batch = tf.convert_to_tensor(x_batch)
y = tf.convert_to_tensor(y)
y_batch = tf.convert_to_tensor(y_batch)

print(f"x_batch shape: {x_batch.shape}") 
print(f"y_batch shape: {y_batch.shape}")
print(f"x shape: {x.shape}") 
print(f"y shape: {y.shape}")

print(f"x_batch dtype: {x_batch.dtype}") 
print(f"y_batch dtype: {y_batch.dtype}")
print(f"x dtype: {x.dtype}") 
print(f"y dtype: {y.dtype}")

In [ ]:
def show_image(img, title):
    plt.imshow(img, 'gray')
    plt.title(title)

idx = 100
show_image(x[idx], LABEL2CLASS[int(y[idx])])

## Generator

In [ ]:
def get_generator(input_dim):

    # input
    input = tf.keras.Input(input_dim)

    # from input_dim -> 7x7xinput_dim
    x = tfl.Dense(units = 7 * 7 * input_dim)(input)
    x = tfl.LeakyReLU(0.2)(x)
    x = tfl.Reshape((7, 7, input_dim))(x)

    # upsampling
    # from 7x7xinput_dim -> 14x14x128
    x = tfl.UpSampling2D()(x)
    x = tfl.Conv2D(128, 5, padding = 'same')(x)
    x = tfl.LeakyReLU(0.2)(x)

    # upsampling
    # from 14x14x128 -> 28x28x128
    x = tfl.UpSampling2D()(x)
    x = tfl.Conv2D(128, 5, padding = 'same')(x)
    x = tfl.LeakyReLU(0.2)(x)

    # conv
    x = tfl.Conv2D(128, 4, padding='same')(x)
    x = tfl.LeakyReLU(0.2)(x)
    x = tfl.Conv2D(128, 4, padding='same')(x)
    x = tfl.LeakyReLU(0.2)(x)
    x = tfl.Conv2D(1, 4, padding = 'same', activation = 'sigmoid')(x)

    # output
    output = x
    
    model = tfm.Model(inputs = input, outputs = output)
    
    return model
    
gen = get_generator(input_dim = LATENT_DIM)

In [ ]:
gen.summary()

In [ ]:
# visualize a random image
random_numbers = np.random.rand(10, LATENT_DIM, CHANNEL)
random_image = gen(random_numbers)
random_image = random_image[5]
show_image(random_image, 'random image')

## Discriminator

In [ ]:
def get_discriminator(input_dim):
    
    input = tf.keras.Input(input_dim)

    x = tfl.Conv2D(32, 5, padding = 'valid')(input)
    x = tfl.LeakyReLU(0.2)(x)
    x = tfl.Dropout(0.4)(x)

    x = tfl.Conv2D(64, 5, padding = 'valid')(x)
    x = tfl.LeakyReLU(0.2)(x)
    x = tfl.Dropout(0.4)(x)


    x = tfl.Conv2D(128, 5, padding = 'valid')(x)
    x = tfl.LeakyReLU(0.2)(x)
    x = tfl.Dropout(0.4)(x)

    x = tfl.Conv2D(256, 5, padding = 'valid')(x)
    x = tfl.LeakyReLU(0.2)(x)
    x = tfl.Dropout(0.4)(x)

    x = tfl.Flatten()(x)
    x = tfl.Dropout(0.4)(x)
    
    output = tfl.Dense(units = 1, activation = 'sigmoid')(x)
    
    model = tfm.Model(inputs = input, outputs = output)
    
    return model

In [ ]:
dis = get_discriminator((HEIGHT, WIDTH, CHANNEL))
dis.summary()

In [ ]:
random_image = tf.expand_dims(random_image, axis = 0)
dis(random_image)

## Training

In [ ]:
# real image -> 0
# fake image -> 1
class GAN(tfm.Model):
    def __init__(self, generator, discriminator, latent_dim, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.gen = generator
        self.dis = discriminator
        self.latent_dim = latent_dim

    def compile(self, g_opt, d_opt, g_loss, d_loss, *args, **kwargs): 
        super().compile(*args, **kwargs)
        self.g_opt = g_opt
        self.d_opt = d_opt
        self.g_loss = g_loss
        self.d_loss = d_loss 

    def train_step(self, image_batch):
        batch_size, height, width, channel = image_batch.shape
        
        # get real and fake images
        real_images = image_batch
        fake_images = self.gen(tf.random.normal((batch_size, self.latent_dim, 1)),  training=False)

        # train dis
        with tf.GradientTape() as d_tape:

            # make predictions
            yhat_real = self.dis(real_images, training = True)
            yhat_fake = self.dis(fake_images, training = True)
            yhat = tf.concat([yhat_real, yhat_fake], axis = 0)

            # correct labels
            y = tf.concat([tf.zeros_like(yhat_real), tf.ones_like(yhat_fake)], axis = 0)

            # Add some noise to the correct labels
            noise_real = 0.15*tf.random.uniform(tf.shape(yhat_real))
            noise_fake = -0.15*tf.random.uniform(tf.shape(yhat_fake))
            y += tf.concat([noise_real, noise_fake], axis=0)
            
            # Calculate loss - BINARYCROSS 
            dis_loss = self.d_loss(y, yhat)

        # apply gradient
        dgrad = d_tape .gradient(dis_loss, self.dis.trainable_variables)
        self.d_opt.apply_gradients(zip(dgrad, self.dis.trainable_variables))

        # train gan
        with tf.GradientTape() as g_tape:
            # generate fake images
            fake_images = self.gen(tf.random.normal((batch_size, self.latent_dim, 1)),  training=True)

            # get predicted labels
            yhat = self.dis(fake_images, training = False)

            # wished labels, we want fake images to be real images
            wished_labels = tf.zeros_like(yhat)

            gen_loss = self.g_loss(yhat, wished_labels)

        # apply gradient
        ggrad = g_tape .gradient(gen_loss, self.gen.trainable_variables)
        self.g_opt.apply_gradients(zip(ggrad, self.gen.trainable_variables))
        
        return {"dis_loss": dis_loss, "gen_loss": gen_loss}

In [ ]:
# generator and discriminator
gen = get_generator(input_dim = LATENT_DIM)
dis = get_discriminator((HEIGHT, WIDTH, CHANNEL))

# define loss 
g_loss = BinaryCrossentropy()
d_loss = BinaryCrossentropy()

# define optimizers
g_opt = Adam(learning_rate = 0.0001)
d_opt = Adam(learning_rate = 0.00001)

gan_model = GAN(gen, dis, LATENT_DIM)
gan_model.compile(g_opt, d_opt, g_loss, d_loss)

d_losses = []
g_losses = []

for epoch in range(EPOCHS):
    print(f"Epoch: {epoch}")
    for batch_idx in progressbar( range(num_of_batches) ):
        losses = gan_model.train_step(x_batch[batch_idx])
        d_losses.append(losses['dis_loss'])
        g_losses.append(losses['gen_loss'])
        if batch_idx == (num_of_batches - 1):
            print(f"Discriminator Loss: {losses['dis_loss']} --- Generator Loss: {losses['gen_loss']}")
    print()